In [ ]:
import pandas as pd
import numpy as np

Read sample messages dataset

In [ ]:
mails_df = pd.read_csv('msgs.csv')
mails_df

Using nltk word_tokenize to convert the message strings into list of words

using stopwords to remove punctuations or irrelevant words from list of words

using PorterStemmer to convert the words into their root form 

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

def process_message(message):
    
    message = message.lower()
    words = word_tokenize(message)
    words = [w for w in words if len(w) > 2]
    
    sw = stopwords.words('english')
    words = [word for word in words if word not in sw]
    
    stemmer = PorterStemmer()
    words = [stemmer.stem(word) for word in words]  
    
    return words

In [ ]:
mails_df['clean_msg'] = mails_df['msg'].apply(process_message)
mails_df

Lets create a word_frequency.json file and start adding each word under ham or spam category in json file with initial count of 1 for new word in file. With repetition of same word, we can increase the count of repititve word under ham or spam caegory

In [ ]:
import json

def word_frequency_category(df):
    cls = df[1]
    json_file = open('word_frequency.json')
    
    word_freq = json.load(json_file)
    for word in df['clean_msg']:
        if word in word_freq['class'][cls]:
            word_freq['class'][cls][word] += 1
        else:
            word_freq['class'][cls][word] = 1
            
    json_file = open('word_frequency.json', 'w')
    json.dump(word_freq, json_file)

In [ ]:
mails_df[['clean_msg', 'class']].apply(word_frequency_category, axis=1)

In [ ]:
mails_df

## p(ham|msg text) = ( p(ham) * p(msg text|ham) ) / p(msg text)

## p(spam|txt) = ( p(spam) * p(msg text|spam) ) / p(msg text)

Since P(msg text) is constant and common in both expressions, we can avoid it. Note that our goal is not to calculate the actual probability but only comparison. 


## P(ham) = no of documents belonging to category ham / total no of documents

## P(spam) = no of documents belonging to category spam / total no of documents

In [ ]:
mails_df.shape

In [ ]:
p_ham = mails_df[mails_df['class']=='ham'].shape[0]/ mails_df.shape[0]
p_spam = mails_df[mails_df['class']=='spam'].shape[0]/ mails_df.shape[0]
p_ham, p_spam

To calculate the above two probabilities, we’ll use wordFrequency file.

P(bodyText | spam) = P(word1 | spam) * P(word2 | spam) * …

P(bodyText | ham) = P(word1 | ham) * P(word2 | ham) * …


To calculate the above two probabilities, we’ll use wordFrequency file. Here word1, word2, word3 up to word-n are total words in bodytext. 

P(word1 | spam) = count of word1 belonging to category spam / total count of words belonging to category spam.

P(word1 | ham) = count of word1 belonging to category ham / total count of words belonging to category ham.

In [ ]:
word_freq = json.load(open('word_frequency.json'))

In [ ]:
spam_words_count = len(word_freq['class']['spam'])
ham_words_count = len(word_freq['class']['ham'])
spam_words_count, ham_words_count

In [ ]:
def ham_spam_msg_prob(msg):
    words = process_message(msg)
    
    ham_prob = 1
    spam_prob = 1
    
    ham_counter=0
    spam_couter=0
#     print(words)
    for word in words:
        print(word)
        if word in word_freq['class']['ham']:
            print('ham prob: ', ((word_freq['class']['ham'][word])/ham_words_count))
            ham_prob *= ((word_freq['class']['ham'][word])/ham_words_count)
            ham_counter += 1
            print('Ham: ', ham_prob)
            
        if word in word_freq['class']['spam']:
            print('spam_prob: ',((word_freq['class']['spam'][word])/spam_words_count))
            spam_prob *= ((word_freq['class']['spam'][word])/spam_words_count)
            spam_couter += 1
            print('Spam: ', spam_prob)
    
    ham_prob = p_ham * ham_prob
    spam_prob = p_spam * spam_prob
    
#     print(ham_prob, spam_prob)
    print(ham_counter, spam_couter)
        
    if ham_counter == 0 and spam_couter == 0:
        print('Cant clasify')
    elif ham_counter == 0 and spam_couter != 0:
        print('Its spam')
        return ham_prob, spam_prob
    elif ham_counter != 0 and spam_couter == 0:
        print('Its not spam')
        return ham_prob, spam_prob
    elif ham_prob > spam_prob:
        print('Its not spam')
    elif ham_prob < spam_prob:
        print('Its spam')
    else:
        print('Cant clasify')


In [ ]:
ham_spam_msg_prob('Free Win cash')